In [3]:
import cv2
import numpy as np
import time
import pandas as pd
from datetime import datetime

cap = cv2.VideoCapture('video2.mp4')

classFile = 'coco.names'
classNames=[]
with open(classFile,'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')

modelConfiguration = 'yolov3.cfg'
modelWeights = 'yolov3 (1).weights'

# modelConfiguration = 'yolov3-tiny.cfg'
# modelWeights = 'yolov3-tiny.weights'


#load the YOLO network model from the harddisk into OpenCV:
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

confthresh=0.5
nmsThreshold=0.3
count_position=550
min_w=70
min_h=70
detect =[]
offset=6 #allowable error between pixel
counter=0
dist=3
                              
def centre_circle(x,y,w,h):
    x1 = int(w/2)
    y1=int(h/2)
    cx = x+x1
    cy = y+y1
    return cx,cy
        
i = 1

while (cap.isOpened()):
    ret, frame = cap.read()
    start_time = time.time()
    frame = frame[:,200:]
    cv2.line(frame,(1, count_position), (1200, count_position), (0,0,255),3)
    blob = cv2.dnn.blobFromImage(frame, 1/255, (320,320),[0,0,0],1, crop = False)
    
    #these two instruction calculate the network response.
    net.setInput(blob)
    #The YOLO neural network has 254 components
    layerNames = net.getLayerNames()
   
    outputNames = [layerNames[i-1] for i in net.getUnconnectedOutLayers()]
    #net.getUnconnectedOutLayers()
    outputs = net.forward(outputNames)
    #function call
    #findobjects(outputs,frame)
    ht,wt,ct = frame.shape
    bbox = []
    classids=[]
    confs=[]
    for output in outputs:
        for det in output:
            scores = det[5:]
            classid = np.argmax(scores)
            conf = scores[classid]
            if conf > confthresh:
                w,h = int(det[2]*wt), int(det[3]*ht)
                x,y = int((det[0]*wt)-w/2),int((det[1]*ht-h/2))
                bbox.append((x,y,w,h))
                classids.append(classid)
                confs.append(float(conf))
    #non max supression
    indices = cv2.dnn.NMSBoxes(bbox, confs, confthresh, nmsThreshold)
    for i in indices:
        box = bbox[i]
        x,y,w,h = box[0],box[1],box[2],box[3]
                
        validate_counter = (w>= min_w) and (h>= min_h)
        if not validate_counter:
            continue
          
        cv2.rectangle(frame,(x,y),(x+w,y+h), (255,0,0),2)
        cv2.putText(frame, f'{classNames[classids[i]].upper()}{int(confs[i]*100)}%',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6,(0,0,255),2 )
        centre = centre_circle(x,y,w,h)
        detect.append(centre)
        cv2.circle(frame, centre, 2, (0,0,255),-1)
        
    for (x,y) in detect:
        if y<(count_position+offset) and y>(count_position-offset):
            counter+=1
            now = datetime.now()
            current_time = now.strftime("%H:%M:%S")
            file = open('count.txt','a')
            file.write("\t"+str(counter))
            file.write("\t\t"+classNames[classids[i]].upper())
            file.close()
        cv2.line(frame,(1, count_position), (1200, count_position), (0,255,0),3)
        detect.remove((x,y))
    cv2.putText(frame,"car counter"+str(counter), (450,70),cv2.FONT_HERSHEY_SIMPLEX, 0.6,(0,0,255),2)
    cv2.imshow("frame",frame)
    if cv2.waitKey(1) & 0xFF== ord('q'):
        break
cap.release()
cv2.destroyAllWindows() 

In [24]:
import cv2
import numpy as np
import time
cap = cv2.VideoCapture("video2.mp4")
cv2.namedWindow('frame',cv2.WINDOW_NORMAL)
p1,p2 = None, None
state=0

def draw(event, x1, y1, flags, param):
    global p1,p2,state
    
    if event == cv2.EVENT_LBUTTONUP:
        if state==0:
            p1 = (x1,y1)
            state+=1
            print("p1",p1[0])
        elif state==1:
            p2 = (x1,y1)
            state+=1
            print("p2",p2)
        
    if event == cv2.EVENT_RBUTTONUP:
        p1,p2 = None, None
        state =0

cv2.setMouseCallback('frame', draw)

while True:
    ret,frame = cap.read()
    if state>1:
        cv2.rectangle(frame,p1,p2,(0,0,255),2)
    cv2.imshow("frame", frame) 
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

p1 699
p2 (833, 478)
p1 474
p2 (544, 512)
